# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
.getOrCreate()

from pyspark.sql.functions import first,split
from pyspark.sql.functions import upper, col
from pyspark.sql.types import StructField, StructType, StringType, LongType, IntegerType
from pyspark.sql.functions import udf, date_format
import datetime as dt
from pyspark.sql.functions import col, countDistinct
from pyspark.sql import functions as F

### Step 1: Scope the Project and Gather Data

#### Scope 
Government Organization wants to keep track on the non immigration trends so as to analyze trend in toursims accross various states and students coming in Unites states for studying.So they want to develop a data model that would help them to observe the cities that students and non immigrants visits or come to study.We will create a star schema of dimensions and fact tables from immigration and US cities demographics to analyze various trends

#### Describe and Gather Data 
The data we are using is I94 Immigration data provided as SAS_data in workspace and US cities demographics under csv file in workspace. 

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

# Start ETL of US CITIES DEMOGRAPHICS DATA SET:
This data looks pretty clean.The exploration steps involve:
1.Looking at a single city. This reveals the grain of the table to be city/state/race. Removing the Race and Count columns gives you duplicate data
2.Seeing that the total of Count for all the races adds up to more than Total Population, indicating people must have been allowed to select more than one race for themselves in the survey.
3.Selecting distinct state codes to be sure that the number was near 50 (to account for 50 states, DC, PR, etc.)

In [15]:
# Read the demographic data here
us_demo=spark.read.csv("./us-cities-demographics.csv", sep=';', header=True)
us_demo.columns

['City',
 'State',
 'Median Age',
 'Male Population',
 'Female Population',
 'Total Population',
 'Number of Veterans',
 'Foreign-born',
 'Average Household Size',
 'State Code',
 'Race',
 'Count']

In [16]:
#check dataset for any data quality issues
us_demo.select("City","state","Median Age","male population","female population","total population", \
                  "foreign-born","Average Household Size").orderBy("city").show()

+-------+----------+----------+---------------+-----------------+----------------+------------+----------------------+
|   City|     state|Median Age|male population|female population|total population|foreign-born|Average Household Size|
+-------+----------+----------+---------------+-----------------+----------------+------------+----------------------+
|Abilene|     Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|
|Abilene|     Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|
|Abilene|     Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|
|Abilene|     Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|
|Abilene|     Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|
|  Akron|      Ohio|      38.1|          96886| 

In [17]:
# Check subset of dataset that maybe causing dupliate rows
us_demo.select("city","state code","Race","count").orderBy("city").show()

+-------+----------+--------------------+------+
|   city|state code|                Race| count|
+-------+----------+--------------------+------+
|Abilene|        TX|American Indian a...|  1813|
|Abilene|        TX|  Hispanic or Latino| 33222|
|Abilene|        TX|               White| 95487|
|Abilene|        TX|               Asian|  2929|
|Abilene|        TX|Black or African-...| 14449|
|  Akron|        OH|               White|129192|
|  Akron|        OH|  Hispanic or Latino|  3684|
|  Akron|        OH|Black or African-...| 66551|
|  Akron|        OH|               Asian|  9033|
|  Akron|        OH|American Indian a...|  1845|
|Alafaya|        FL|  Hispanic or Latino| 34897|
|Alafaya|        FL|               Asian| 10336|
|Alafaya|        FL|               White| 63666|
|Alafaya|        FL|Black or African-...|  6577|
|Alameda|        CA|               White| 44232|
|Alameda|        CA|American Indian a...|  1329|
|Alameda|        CA|Black or African-...|  7364|
|Alameda|        CA|

#### CLEANING DATASET:
1. 'Race' and 'Count' records are causing duplicate rows. We will separate them from US demographics data set and include 'City' and 'State Code'.
2. 'Race' will be pivoted to column headers and saved to us_race_cnt dataset
3. US_demo dataset will be cleaned of duplicate rows (shown above).
4. Cleaned US_demo dataset will be joined back with us_race_cnt dataset to eventually have unique row

In [18]:
# Creating new columns from Race using Pivot and aggregating using count
us_race_cnt=(us_demo.select("city","state code","Race","count")
    .groupby(us_demo.City, "state code")
    .pivot("Race")
    .agg(first("Count")))


In [19]:
# Checking dataset
us_race_cnt.orderBy("city").show()

+------------+----------+---------------------------------+-----+-------------------------+------------------+------+
|        City|state code|American Indian and Alaska Native|Asian|Black or African-American|Hispanic or Latino| White|
+------------+----------+---------------------------------+-----+-------------------------+------------------+------+
|     Abilene|        TX|                             1813| 2929|                    14449|             33222| 95487|
|       Akron|        OH|                             1845| 9033|                    66551|              3684|129192|
|     Alafaya|        FL|                             null|10336|                     6577|             34897| 63666|
|     Alameda|        CA|                             1329|27984|                     7364|              8265| 44232|
|      Albany|        NY|                             1611| 8090|                    31303|              9368| 58368|
|      Albany|        GA|                              4

In [20]:
# Drop columns we don't need and drop duplicate rows
uscols=["Number of Veterans","Race","Count"]
us_demo=us_demo.drop(*uscols).dropDuplicates()

In [21]:
# Comparing row count between original and new dataset with dropped duplicate rows
(us_demo.count(),us_race_cnt.count())

(596, 596)

In [22]:
# Joining us_demo and us_race-cnt on common city,state and code
us_demo=us_demo.join(us_race_cnt, ["city","state code"])
us_demo.show(5)

+---------------+----------+-----------+----------+---------------+-----------------+----------------+------------+----------------------+---------------------------------+-----+-------------------------+------------------+------+
|           City|State Code|      State|Median Age|Male Population|Female Population|Total Population|Foreign-born|Average Household Size|American Indian and Alaska Native|Asian|Black or African-American|Hispanic or Latino| White|
+---------------+----------+-----------+----------+---------------+-----------------+----------------+------------+----------------------+---------------------------------+-----+-------------------------+------------------+------+
|Highlands Ranch|        CO|   Colorado|      39.6|          49186|            53281|          102467|        8827|                  2.72|                             1480| 5650|                     1779|              8393| 94499|
|           Kent|        WA| Washington|      33.4|          61825|         

In [23]:
# Change `state code` column name to `state_code` and other similar problems to avoid parquet complications
us_demo=us_demo.select('City', col('State Code').alias('State_Code'), 'State', col('Median Age').alias('Median_age'),
     col('Male Population').alias('Male_Population'), col('Female Population').alias('Female_Population'), 
        col('Total Population').alias('Total_Population'), 'Foreign-born', 
          col('Average Household Size').alias('Average_Household_Size'),
             col('American Indian and Alaska Native').alias('American_Indian_and_Alaska_Native_Pop'), 
                 col('Asian').alias('Asian_Pop'), 
                    col('Black or African-American').alias('Black_or_AfricanAmerican_Pop'), 
                      col('Hispanic or Latino').alias('Hispanic_or_Latino_Pop'), 
                        col('White').alias('White_Pop'))

In [24]:
us_demo.show(2)

+---------------+----------+----------+----------+---------------+-----------------+----------------+------------+----------------------+-------------------------------------+---------+----------------------------+----------------------+---------+
|           City|State_Code|     State|Median_age|Male_Population|Female_Population|Total_Population|Foreign-born|Average_Household_Size|American_Indian_and_Alaska_Native_Pop|Asian_Pop|Black_or_AfricanAmerican_Pop|Hispanic_or_Latino_Pop|White_Pop|
+---------------+----------+----------+----------+---------------+-----------------+----------------+------------+----------------------+-------------------------------------+---------+----------------------------+----------------------+---------+
|Highlands Ranch|        CO|  Colorado|      39.6|          49186|            53281|          102467|        8827|                  2.72|                                 1480|     5650|                        1779|                  8393|    94499|
|       

In [25]:
# Drop the `state` column
us_demo=us_demo.drop("state")

In [27]:
# Now write (and overwrite) transformed `US_demo` dataset onto parquet file(In data folder)
us_demo.write.mode('overwrite').parquet("./data/us_cities_demographics.parquet")

# Start ETL of i94 non-immigration dataset :

In [51]:
# Read i94 non-immigration dataset
i94_immigration=spark.read.parquet("sas_data")

In [52]:
i94_immigration.columns

['cicid',
 'i94yr',
 'i94mon',
 'i94cit',
 'i94res',
 'i94port',
 'arrdate',
 'i94mode',
 'i94addr',
 'depdate',
 'i94bir',
 'i94visa',
 'count',
 'dtadfile',
 'visapost',
 'occup',
 'entdepa',
 'entdepd',
 'entdepu',
 'matflag',
 'biryear',
 'dtaddto',
 'gender',
 'insnum',
 'airline',
 'admnum',
 'fltno',
 'visatype']

In [53]:
i94_immigration.show()

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
|5748518.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|  32.0|    1.0|  

## we are taking into consideration the following columns as it makes sense and omitting the remaining columns.This will be basis of our facts tables

In [54]:
#Convert columns reads as string/doublr to integer
i94_immigration=i94_immigration.select(col("i94res").cast(IntegerType()),col("i94port"),\
                           col("cicid").cast(IntegerType()),
                           col("arrdate").cast(IntegerType()), \
                           col("i94mode").cast(IntegerType()),col("depdate").cast(IntegerType()),
                           col("i94bir").cast(IntegerType()),col("i94visa").cast(IntegerType()), 
                           col("count").cast(IntegerType()), \
                              "gender","fltno")


In [32]:
#Check for duplicate rows on each dataset by comparing original total rows with .dropDuplicates()
i94_immigration.count(), i94_immigration.dropDuplicates().count()

(3096313, 3096313)

In [77]:
i94_immigration.show(3)

+------+-------+-------+-------+-------+-------+------+-------+-----+------+-----+---+----------+----------+----+----------+----------+---------------+-----------------+----------------+------------+----------------------+-------------------------------------+---------+----------------------------+----------------------+---------+------------+
|i94res|i94port|  cicid|arrdate|i94mode|depdate|i94bir|i94visa|count|gender|fltno| id| port_city|port_state|City|State_Code|Median_age|Male_Population|Female_Population|Total_Population|Foreign-born|Average_Household_Size|American_Indian_and_Alaska_Native_Pop|Asian_Pop|Black_or_AfricanAmerican_Pop|Hispanic_or_Latino_Pop|White_Pop|arrival_date|
+------+-------+-------+-------+-------+-------+------+-------+-----+------+-----+---+----------+----------+----+----------+----------+---------------+-----------------+----------------+------------+----------------------+-------------------------------------+---------+----------------------------+---------

#### I have transformed I94_SAS-Labels_Description into I-94ADDR.csv,I94CIT_I94RES.csv,I94MODE.csv,I94Visa.csv and finally saving them into respective parquet files.

## ETL I94Mode 

In [34]:
# Create i94mode list
i94mode=spark.read.csv('./I94MODE.csv',header=True)
i94mode.show()
# Create i94mode parquet file
i94mode.write.mode("overwrite").parquet('./data/i94mode.parquet')

+---+------------+
| ID|        Mode|
+---+------------+
|  1|         Air|
|  2|         Sea|
|  3|        Land|
|  9|Not reported|
+---+------------+



## ETL I94Port 

In [35]:
# Read i94port CSV file
i94port= pd.read_csv('./I94PORT.csv')
new=i94port['Port'].str.split(",",expand=True)
i94port['Port_city']=new[0]
i94port['Port_state']=new[1]
i94port.drop(columns=['Port'],inplace=True)
i94port.head()

,ID,Port_city,Port_state
0,ALC,ALCAN,AK
1,ANC,ANCHORAGE,AK
2,BAR,BAKER AAF - BAKER ISLAND,AK
3,DAC,DALTONS CACHE,AK
4,PIZ,DEW STATION PT LAY DEW,AK


In [36]:
# Now convert pd dataframe to spark dataframe
# Create a schema for the dataframe
i94port_schema = StructType([
    StructField('id', StringType(), True),
    StructField('port_city', StringType(), True),
    StructField('port_state', StringType(), True)
])
i94port=spark.createDataFrame(i94port, i94port_schema)

In [37]:
# Create parquet file
i94port.write.mode('overwrite').parquet('./data/i94port.parquet')

## ETL I94Res

In [38]:
#Read I94CIT_I94RES.csv 
i94res=spark.read.csv('./I94CIT_I94RES.csv',header=True)
i94res=i94res.withColumnRenamed('I94CTRY','country')
i94res.show()

+----+---------------+
|Code|        country|
+----+---------------+
| 582|         MEXICO|
| 236|    AFGHANISTAN|
| 101|        ALBANIA|
| 316|        ALGERIA|
| 102|        ANDORRA|
| 324|         ANGOLA|
| 529|       ANGUILLA|
| 518|ANTIGUA-BARBUDA|
| 687|      ARGENTINA|
| 151|        ARMENIA|
| 532|          ARUBA|
| 438|      AUSTRALIA|
| 103|        AUSTRIA|
| 152|     AZERBAIJAN|
| 512|        BAHAMAS|
| 298|        BAHRAIN|
| 274|     BANGLADESH|
| 513|       BARBADOS|
| 104|        BELGIUM|
| 581|         BELIZE|
+----+---------------+
only showing top 20 rows



In [39]:
# Create parquet file
i94res.write.mode('overwrite').parquet('./data/i94res.parquet')

## ETL I94Visa

In [78]:
#read I94 Visa
i94visa=spark.read.csv('./I94VISA.csv',header=True)
i94visa.columns

['ID', 'Type']

In [41]:
# Create parquet file
i94visa.write.mode('overwrite').parquet('./data/i94visa.parquet')

#### Joining I94_immigration and I94Port(left join)

In [55]:
# Add i94port city and state columns to i94 dataframe
i94_immigration=i94_immigration.join(i94port, i94_immigration.i94port==i94port.id, how='left')

In [56]:
i94_immigration.show()

+------+-------+-------+-------+-------+-------+------+-------+-----+------+-----+---+---------+----------+
|i94res|i94port|  cicid|arrdate|i94mode|depdate|i94bir|i94visa|count|gender|fltno| id|port_city|port_state|
+------+-------+-------+-------+-------+-------+------+-------+-----+------+-----+---+---------+----------+
|   297|    BGM|5761355|  20574|      1|   null|    63|      1|    1|     F|00812|BGM|   BANGOR|        ME|
|   297|    BGM|5761356|  20574|      1|  20674|    43|      1|    1|     M|00812|BGM|   BANGOR|        ME|
|   297|    BGM|5761357|  20574|      1|  20685|    48|      1|    1|     M|00812|BGM|   BANGOR|        ME|
|   135|    BGM| 460085|  20547|      1|  20550|    34|      2|    1|     M|EDC10|BGM|   BANGOR|        ME|
|   135|    BGM| 473180|  20547|      1|  20687|    69|      1|    1|     M|EDC92|BGM|   BANGOR|        ME|
|   135|    BGM| 473264|  20547|      1|   null|    64|      1|    1|     M|EDC92|BGM|   BANGOR|        ME|
|   135|    BGM| 473265|  20

In [58]:
us_demo.columns

['City',
 'State_Code',
 'Median_age',
 'Male_Population',
 'Female_Population',
 'Total_Population',
 'Foreign-born',
 'Average_Household_Size',
 'American_Indian_and_Alaska_Native_Pop',
 'Asian_Pop',
 'Black_or_AfricanAmerican_Pop',
 'Hispanic_or_Latino_Pop',
 'White_Pop']

In [60]:
# Join US with i94_spark to get fact table `i94non_immigrant_port_entry`
# NOTE: We use left join againt city records which may cause null values because
# we may not currently have demographic stats on all U.S. ports of entry
i94_immigration=i94_immigration.join(us_demo, (upper(i94_immigration.port_city)==upper(us_demo.City)) & \
                                           (upper(i94_immigration.port_state)==upper(us_demo.State_Code)), how='left')

In [62]:
i94_immigration.count()

3096313

In [63]:
i94_immigration.show(5)

+------+-------+-------+-------+-------+-------+------+-------+-----+------+-----+---+----------+----------+----+----------+----------+---------------+-----------------+----------------+------------+----------------------+-------------------------------------+---------+----------------------------+----------------------+---------+
|i94res|i94port|  cicid|arrdate|i94mode|depdate|i94bir|i94visa|count|gender|fltno| id| port_city|port_state|City|State_Code|Median_age|Male_Population|Female_Population|Total_Population|Foreign-born|Average_Household_Size|American_Indian_and_Alaska_Native_Pop|Asian_Pop|Black_or_AfricanAmerican_Pop|Hispanic_or_Latino_Pop|White_Pop|
+------+-------+-------+-------+-------+-------+------+-------+-----+------+-----+---+----------+----------+----+----------+----------+---------------+-----------------+----------------+------------+----------------------+-------------------------------------+---------+----------------------------+----------------------+---------+
|

In [65]:
i94_immigration.columns

['i94res',
 'i94port',
 'cicid',
 'arrdate',
 'i94mode',
 'depdate',
 'i94bir',
 'i94visa',
 'count',
 'gender',
 'fltno',
 'id',
 'port_city',
 'port_state',
 'City',
 'State_Code',
 'Median_age',
 'Male_Population',
 'Female_Population',
 'Total_Population',
 'Foreign-born',
 'Average_Household_Size',
 'American_Indian_and_Alaska_Native_Pop',
 'Asian_Pop',
 'Black_or_AfricanAmerican_Pop',
 'Hispanic_or_Latino_Pop',
 'White_Pop']

In [67]:
# Convert SAS arrival date to datetime format
get_date = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(x)).isoformat() if x else None)
i94_immigration= i94_immigration.withColumn("arrival_date", get_date(i94_immigration.arrdate))

In [79]:
i94_immigration=i94_immigration.withColumnRenamed('i94res','res_id')\
                               .withColumnRenamed('i94port','port_id')\
                               .withColumnRenamed('i94mode','mode_id')\
                               .withColumnRenamed('i94visa','visa_id')

In [80]:
i94_immigration.show(2)

+------+-------+-------+-------+-------+-------+------+-------+-----+------+-----+---+----------+----------+----+----------+----------+---------------+-----------------+----------------+------------+----------------------+-------------------------------------+---------+----------------------------+----------------------+---------+------------+
|res_id|port_id|  cicid|arrdate|mode_id|depdate|i94bir|visa_id|count|gender|fltno| id| port_city|port_state|City|State_Code|Median_age|Male_Population|Female_Population|Total_Population|Foreign-born|Average_Household_Size|American_Indian_and_Alaska_Native_Pop|Asian_Pop|Black_or_AfricanAmerican_Pop|Hispanic_or_Latino_Pop|White_Pop|arrival_date|
+------+-------+-------+-------+-------+-------+------+-------+-----+------+-----+---+----------+----------+----+----------+----------+---------------+-----------------+----------------+------------+----------------------+-------------------------------------+---------+----------------------------+---------

In [81]:
i94_immigration.drop('arrdate').write.mode("overwrite").parquet('./data/i94_immigration.parquet')

## Date ETL

In [74]:
i94date=i94_immigration.select(
                                col('arrival_date'),
                                date_format('arrival_date','M').alias('arrival_month'),
                                date_format('arrival_date','E').alias('arrival_dayofweek'), 
                                date_format('arrival_date', 'y').alias('arrival_year'), 
                                date_format('arrival_date', 'd').alias('arrival_day'),
                                date_format('arrival_date','w').alias('arrival_weekofyear')).dropDuplicates()

In [75]:
i94date.show(5)

+------------+-------------+-----------------+------------+-----------+------------------+
|arrival_date|arrival_month|arrival_dayofweek|arrival_year|arrival_day|arrival_weekofyear|
+------------+-------------+-----------------+------------+-----------+------------------+
|  2016-04-05|            4|              Tue|        2016|          5|                15|
|  2016-04-29|            4|              Fri|        2016|         29|                18|
|  2016-04-20|            4|              Wed|        2016|         20|                17|
|  2016-04-17|            4|              Sun|        2016|         17|                17|
|  2016-04-12|            4|              Tue|        2016|         12|                16|
+------------+-------------+-----------------+------------+-----------+------------------+
only showing top 5 rows



In [76]:
i94date.write.mode("overwrite").partitionBy("arrival_year", "arrival_month").parquet('./data/i94date.parquet')

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
#### Map out the conceptual data model and explain why you chose that model



#### We are selecting star schema with i94_immigrations as facts table and i94date,i94mode,i94port,i94res,i94visa and us_cities demographics as dimensions tables.As we have stored variuos dimensions and facts tables in parquet files under data in workspace and in S3 buckets (AWS)  we can easily implement them in AWS Redshift.We create the schema by running the SQL script found in create_tables.sql. From there, our model is ready to be explored by the customers whether through open query editor in Redshift itself or using a dashboard tool such as Tableau or Power BI..We are storing data in its lowest granularity as to allow complex queries with better performance and flexibility.

#### 3.2 Mapping Out Data Pipelines
#### To accomplish all task related to preprocessing of datasets I have used etl.py and etl1.py.There you will find different functions to load,select,clean,transform and store results in datasets in a convinent way.The open-source framework Apache Spark was the main tool in this journey.

## Follwing are the steps to pipeline the data into model
1. Create the dimensions (i94port, i94visa, i94res, i94mode) from i94_SAS_Labels_Descriptions.SAS file. *NOTE: Once they're created it does not have to be included in future Data Pipeline schedules because these are essentially master records which do not frequently get added or changed on the dimension tables.
2.  US Cities Demo dataset file to form us_demo dataframe
3. Create 'us_race_cnt' from us_demo
4. Write (and overwrite) transformed US_cities_demographics onto parquet file
5. Read i94port dimension parquet file so we can use it to join with i94_immigration. This will add i94port city and state columns to i94_immigration
6. join i94_immigration and US_demo to form fact i94_immigration 
7. time dimension from i94_immigration and save to parquet file.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

## The data pipelines is built inside etl.py and etl1.py

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [84]:
# Perform quality checks here
if us_demo.count() > 0:
    print('Passed reading data file.')
else:
    print('Seems to be nothing in file!')

if us_demo.count() == us_race_cnt.count():
    print('Transformation went perfect.')
else:
    print('Inconsistant data between both dataframes!')

if i94_immigration.count() > 0:
    print('Passed reading data file.')
else:
    print('Seems to be nothing in file!')
    


Passed reading data file.
Transformation went perfect.
Passed reading data file.


## 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.


'''

## i94date
#### arrival_date Date PK  :   Non-immigrant arrival date in the USA 
#### arrival_month int    :   Non-immigrant arrival month
#### arrival_day int      :   Non-immigrant arrival day
#### arrival_year int     :   Non-immigrant arrival year
#### arrival_week int     :   Non immigrant week of year

##   i94port
#### id string PK         : 3 character code of destination USA city
#### port_city varchar    : city name
#### port_state varchar   : state name

##   i94visa
#### id int PK            : visa code number
#### type string          : Type of visa

##   i94res
#### code int PK          : 3 digit code of nationality 
#### country varchar      : country of Immigrant

##   i94mode
#### id int PK            : 1 digit code (plane, boat, etc) of travel 
#### transport varchar    : Mode of transport according to code

##   i94_immigrant
#### cicid int PK         : ID that uniquely identify one record in the dataset
#### arrival_date date FK : arrival date in the USA
#### depdate int          : Departure Date from the USA
#### port_id string FK    : i94port.id 3 character code of destination USA city from i94 non-immigration data
#### visa_id int FK       : i94visa.id reason for immigration from i94 non-immigration data
#### res_id int FK        : i94res.id 3 digit code of nationality from i94 non-immigration data
#### mode_id int FK       : i94mode.id 1 digit mode (plane, boat, etc) of travel code from i94 non-immigration data
#### age int              : Age of non-immigrant in years from i94 non-immigration data
#### gender string        : Non-immigrant sex from i94 non-immigration data
#### cnt_of_one int       : count of one per row used for statistical metrics from i94 non-immigration data
#### median_age float     : Median age of population in city and state from US cities demographics data
#### male_pop int         : Male population of city and state from US cities demographics data
#### female_pop int       : Female population of city and state from US cities demographics data
#### total_pop int          : Total population of city and state from US cities demographics data
#### foreign_born_pop int : Foreign born population of city and state from US cities demographics data
#### avg_household_size float :Average household size of city and state from US cities demographics data
#### american_indian_alaskan_native_pop int : Aerican Indian population of city and state from US cities demographics data
#### asian_pop int        : Asian population of city and state from US cities demographics data
#### black_african_american_pop int : Black population of city and state from US cities demographics data
#### hispanic_pop int     : Hispanic population of city and state from US cities demographics data
#### white_pop int        : White population of city and state from US cities demographics data

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
The whole solution implemented here is mounted on top of cloud computing technology, AWS in particular. Because the cloud computing provides a low-cost, scalable, and highly reliable infrastructure platform in the cloud this is a natural choice for every new solution like we did here. Every service we use (S3, EMR, Redshift) has reasonable cost and is ‘pay as you go’ pricing. So we can start small and scale as our solution grows. No up-front costs involved.

* Propose how often the data should be updated and why.
1.  I94_immigrant and date dimension need to be updated monthly
2.  The US Cities Demographics data is updated every ten years according to https://www.usa.gov/statistics. So, the new US Cities Demographics data set maybe coming after year 2020. And may need updating after one year or two years as of 2019.


* The data was increased by 100x.
Deploy this Spark solution on a cluster using AWS (EMR cluster) and use S3 for data and parquet file storage. AWS will easily scale when data increases by 100x

* The data populates a dashboard that must be updated on a daily basis by 7am every day.

* The database needed to be accessed by 100+ people.
There should be no problem with 100 or so people accessing this data. However, the date-partitioned nature of the solution proposed above would also help in this case. If access by multiple users continues to be a problem you can mitigate that by having the data replicate to different nodes used by different users. If your users are located around the world, a replication node near each group of people would be best.